In [10]:
# import subprocess
# import sys

# def install_missing_modules():
#     required_modules = ['numpy', 'pandas', 'matplotlib', 'plotly', 'ipywidgets', 'scipy']
    
#     try:
#         import pip
#     except ImportError:
#         print("pip is not installed. Please install pip and try again.")
#         return
    
#     for module in required_modules:
#         try:
#             __import__(module)
#         except ImportError:
#             print(f"Module '{module}' is missing. Installing now...")
#             subprocess.check_call([sys.executable, '-m', 'pip', 'install', module])
#             print(f"Module '{module}' has been installed successfully.")

# # Call the function to install missing modules
# install_missing_modules()

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import datetime
import re
# import plotly.graph_objects as go
# import plotly.express as px
# from plotly.subplots import make_subplots
# from plotly.offline import init_notebook_mode, iplot
# import ipywidgets as widgets
# from ipywidgets import interact_manual, IntSlider, FloatSlider, interact, interactive, fixed
# import kaleido
# from scipy import ndimage
# from plotly.graph_objs import FigureWidget


# %matplotlib widget
# %matplotlib inline

# pd.set_option('display.max_rows', 40)
# pd.set_option('display.max_columns', 40)

# init_notebook_mode(connected=True)

In [2]:

# Read the CSV file with specified header
data = pd.read_csv('/mnt/c/Users/John/Documents/GitHub/Grid_Analysis/Uk TEMPERATURE MEANS.csv')

In [3]:
# Check that the dates in all date columns are consistent
assert all(data['date'].eq(data['date.1']) & data['date'].eq(data['date.2'])), "Dates are not consistent across columns"

# Set the index to the first date column
data = data.set_index('date')

# Drop the redundant date columns
data = data.drop(columns=['date.1', 'date.2'])

print(data)



            Temp_mean  Temp_min  Temp_max
date                                     
1878-01-01        3.5       0.9       6.2
1878-01-02        5.5       3.3       7.8
1878-01-03        7.0       5.3       8.8
1878-01-04        7.2       6.0       8.4
1878-01-05        6.6       5.6       7.7
...               ...       ...       ...
2023-04-22        8.2       3.2      13.3
2023-04-23        9.7       5.9      13.5
2023-04-24        7.1       4.5       9.6
2023-04-25        5.0      -0.8      10.8
2023-04-26        7.1       2.6      11.6

[53076 rows x 3 columns]


In [4]:


# Assuming 'data' is your dataframe
data = data.loc[data.index >= '2016-01-01']

# Create a new empty dataframe for temperature values with timestamps
temp_data_with_timestamps = pd.DataFrame(columns=['DateTime', 'Temp'])

# Loop through each day in the data dataframe
for date, row in data.iterrows():
    # Convert 'date' to datetime.date
    date = pd.to_datetime(date).date()

    # Temp_min occurs between 2 AM and 4 AM
    temp_min_hours = [2, 3, 4]
    for temp_min_hour in temp_min_hours:
        temp_min_timestamp = datetime.datetime.combine(date, datetime.time(temp_min_hour, 0))
        temp_min_data = pd.DataFrame({"DateTime": [temp_min_timestamp], "Temp": [row["Temp_min"]]})
        temp_data_with_timestamps = pd.concat([temp_data_with_timestamps, temp_min_data], ignore_index=True)

    # Temp_max occurs between 1 PM and 3 PM
    temp_max_hours = [13, 14, 15]
    for temp_max_hour in temp_max_hours:
        temp_max_timestamp = datetime.datetime.combine(date, datetime.time(temp_max_hour, 0))
        temp_max_data = pd.DataFrame({"DateTime": [temp_max_timestamp], "Temp": [row["Temp_max"]]})
        temp_data_with_timestamps = pd.concat([temp_data_with_timestamps, temp_max_data], ignore_index=True)

# Set the DateTime as index
temp_data_with_timestamps = temp_data_with_timestamps.set_index("DateTime")

print(temp_data_with_timestamps.head(10))


/tmp/ipykernel_52053/3323082822.py:17: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



                     Temp
DateTime                 
2016-01-01 02:00:00   0.4
2016-01-01 03:00:00   0.4
2016-01-01 04:00:00   0.4
2016-01-01 13:00:00   8.9
2016-01-01 14:00:00   8.9
2016-01-01 15:00:00   8.9
2016-01-02 02:00:00   1.9
2016-01-02 03:00:00   1.9
2016-01-02 04:00:00   1.9
2016-01-02 13:00:00  10.0


In [5]:
# Assuming 'data' is your temp_data_with_timestamps dataframe
data = temp_data_with_timestamps

# Find the start and end dates
start_date = data.index.min()
end_date = data.index.max() + pd.DateOffset(hours=1)

# Generate a new DatetimeIndex with hourly frequency
new_index = pd.date_range(start=start_date, end=end_date, freq='H')

# Reindex the dataframe and fill missing values with NaN
data_reindexed = data.reindex(new_index)
data_interpolated = data_reindexed

# Interpolate missing values using cubic method
data_interpolated.interpolate(method='quadratic', inplace=True, )

data_interpolated['Temp'] =data_interpolated['Temp'].round(2)


In [6]:
data_interpolated
data_filtered = data_interpolated

In [7]:
def heating_on_temp_and_months(df, heating_on_temperature, start_month, end_month):

    """
    Calculates the temperature difference between the outside temperature and the heating threshold.

    Args:
        df (pandas.DataFrame): The input DataFrame containing temperature data.
        heating_on_temperature (float): The temperature threshold at which heating is turned on.
        start_month (int): The starting month of the heating season (inclusive).
        end_month (int): The ending month of the heating season (inclusive).

    Returns:
        pandas.DataFrame: The modified DataFrame with the temperature difference calculated.
    """
    ...
          
    # Calculate the temperature difference with respect to the threshold
    df['Temp_diff'] = heating_on_temperature - df['Temp']
    
    # Set the output to zero if Temp is greater than the threshold
    df.loc[df['Temp'] > heating_on_temperature, 'Temp_diff'] = 0
    
    # Set values between the end month and the start month to zero
    df.loc[(df.index.month > end_month) & (df.index.month < start_month), 'Temp_diff'] = 0
    return df

In [8]:
def apply_heating_season(df, heating_on_temperature=17.0, start_month=9, end_month=6):
    """
    Applies the heating season adjustments to the input DataFrame.
    Args:
        df (pandas.DataFrame): The input DataFrame containing temperature data.
        heating_on_temperature (float, optional): The temperature threshold at which heating is turned on. Default is 17.0.
        start_month (int, optional): The starting month of the heating season (inclusive). Default is 9.
        end_month (int, optional): The ending month of the heating season (inclusive). Default is 6.
    Returns:
        pandas.DataFrame: The modified DataFrame with the temperature difference calculated.
    """
    # Apply the heating season adjustments and retrieve the modified data
    adjusted_data = heating_on_temp_and_months(df, heating_on_temperature, start_month, end_month)
    
    # Return the updated DataFrame
    return adjusted_data

# Call the function with your DataFrame and capture the returned DataFrame
filtered_data = apply_heating_season(data_filtered)

In [9]:
import pandas as pd

# Constants
START_DATE = '2016-06-01'
END_DATE = '2022-06-01'
AVERAGE_HEATING_USE_KWH = 12000
YEARS_MODELLED = 6

# Fill missing values in the 'Temperature_diff_with_setback' column using backward fill
filtered_data['Temperature_diff_with_setback'] = filtered_data['Temp_diff']
filtered_data['Temperature_diff_with_setback'] = filtered_data['Temperature_diff_with_setback'].bfill()

# Filter data for the specified period
filtered_data_for_load_calculation = filtered_data[(filtered_data.index >= START_DATE) & (filtered_data.index <= END_DATE)]

# Calculate cumulative temperature difference with setback for the specified period
cumulative_temp_diff_with_setback = filtered_data_for_load_calculation['Temperature_diff_with_setback'].cumsum()

# Calculate the total heating use over the period
total_heating_over_period = YEARS_MODELLED * AVERAGE_HEATING_USE_KWH

# Calculate the heating load per temperature difference
heating_load_per_temp_diff = total_heating_over_period / cumulative_temp_diff_with_setback.max()

# Calculate the hourly heating load by multiplying the temperature difference with setback by the heating load per temperature difference
filtered_data['hourly_heating_load_hour_kWh'] = filtered_data['Temperature_diff_with_setback'] * heating_load_per_temp_diff

The code is used to calculate the heating load for a typical UK household based on the temperature difference between a threshold temperature and the external temperature, adjusted with a setback value to reduce demand at certain times. The calculations are performed using the following steps:

    The filtered_data['Temperature_diff_with_setback'] column is filled with the temperature difference values adjusted with the setback. The bfill() function is used to backward fill any missing values.
    The data is filtered for a specified period from June 1, 2016, to June 1, 2022, and stored in the filtered_data_temp_load_calc variable.
    The cumulative temperature difference for the specified period is calculated using the cumsum() function on the 'Temperature_diff_with_setback' column. This gives the total sum of the temperature differences over the entire period.
    The average yearly heating use for a UK household is defined as 12,000 kilowatt-hours (kWh).
    The number of years modeled is set to 6.
    The average heating use over the 6-year period is calculated by multiplying the average yearly heating use (12,000 kWh) by the number of years modeled (6).
    The heating and temperature ratio is calculated by dividing the average heating use over the period by the maximum cumulative temperature difference. This ratio represents the amount of heating energy required per unit of temperature difference.
    The 'hourly_heating_load_hour_kWh' column is created by multiplying the 'Temperature_diff_with_setback' column with the heating and temperature ratio. This normalizes the heating load based on the temperature difference and the average heating use.

The theory behind these calculations is that if the outside temperature is known and the average gas use of a UK household is known, then each unit of energy used is related to the outside temperature and the duration of that temperature.

By summing the hourly temperature differences across the timeframe, we get the total temperature difference over the period. By calculating the average energy usage of a UK household over the same period, we can determine the relationship between the temperature difference and the energy consumption.

Dividing the total energy used (in kWh) by the summed hourly temperature differences gives us a ratio that represents the proportion of a UK household's annual energy usage per unit of temperature difference.

This ratio allows us to estimate the heating load for a typical UK household based on the temperature difference at any given time. By multiplying the temperature difference with the heating and temperature ratio, we obtain the normalized heating load.

These calculations provide a way to convert the temperature difference into a heating load, taking into account the average energy consumption of a UK household over a specific period.

In [10]:
def apply_linear_cop(df, column_in, column_out,  min_temp=-5, max_temp=20, min_cop=3.5, max_cop=7.5):
    # Calculate the slope and intercept of the linear relationship
    slope = (max_cop - min_cop) / (max_temp - min_temp)
    intercept = min_cop - slope * min_temp

    # Define the linear function
    def linear_cop(temp):
        if temp <= min_temp:
            return min_cop
        elif temp >= max_temp:
            return max_cop
        else:
            return slope * temp + intercept

    # Apply the linear function to the specified column
    df[column_out] = df[column_in].apply(linear_cop)

    return df
    
#apply_linear_cop(filtered_data, 'Temp', 'COP' )

In [11]:
def interactive_apply_linear_cop(df, min_temp=-5, max_temp=20, min_cop=3.5, max_cop=7.5, num_of_heatpumps_millions=1):
    # Apply linear COP calculation to the existing filtered_data_1 DataFrame
    filtered_data_1_updated = apply_linear_cop(df, 'Temp', 'COP', min_temp, max_temp, min_cop, max_cop)
    filtered_data_1_updated['Electricity_load'] = filtered_data_1_updated['hourly_heating_load_hour_kWh'] / filtered_data_1_updated['COP']
    filtered_data_1_updated['Electricity_load_total'] = filtered_data_1_updated['Electricity_load'] * num_of_heatpumps_millions

    # Create a FigureWidget for interactive plotting
    fig = FigureWidget()

    # Add traces to the FigureWidget
    fig.add_trace(go.Scatter(x=filtered_data_1_updated.index, y=filtered_data_1_updated['Electricity_load_total'], mode='lines', name='Electricity Grid Load (GW)'))
    fig.add_trace(go.Scatter(x=df.index, y=df['Temp'], mode='lines', name='Temp'))
    fig.add_trace(go.Scatter(x=df.index, y=df['Temperature_diff_with_setback'], mode='lines', name='Temperature_diff_with_setback'))
    fig.add_trace(go.Scatter(x=df.index, y=df['hourly_heating_load_hour_kWh'], mode='lines', name='hourly_heating_load_hour_kWh'))

    # Update the layout of the FigureWidget
    fig.update_layout(title='Electricity Load (GW)', xaxis_title='Time', yaxis_title='Electricity Load')

    # Define the update function for the interactive plot
    def update_plot(min_temp, max_temp, min_cop, max_cop, num_of_heatpumps_millions):
        # Apply linear COP calculation with the updated parameters
        nonlocal filtered_data_1_updated
        filtered_data_1_updated = apply_linear_cop(df, 'Temp', 'COP', min_temp, max_temp, min_cop, max_cop)
        filtered_data_1_updated['Electricity_load'] = filtered_data_1_updated['hourly_heating_load_hour_kWh'] / filtered_data_1_updated['COP']
        filtered_data_1_updated['Electricity_load_total'] = filtered_data_1_updated['Electricity_load'] * num_of_heatpumps_millions

        # Update the traces with the new data
        fig.data[0].y = filtered_data_1_updated['Electricity_load_total']

        # Update the yearly statistics table
        yearly_stats = filtered_data_1_updated.groupby(filtered_data_1_updated.index.year).agg({
            'hourly_heating_load_hour_kWh': ['max', 'mean', 'sum'],
            'Electricity_load': ['max', 'mean', 'sum']
        })
        yearly_stats.columns = ['_'.join(col) for col in yearly_stats.columns]
        display(yearly_stats)

    # Create widgets for the function parameters
    style={'description_width': 'initial'}
    layout=widgets.Layout(width='500px')
    interactive_plot_widget = widgets.interactive(
        update_plot,
        min_temp=widgets.IntSlider(min=-5, max=20, step=1, value=-5, style=style, layout=layout),
        max_temp=widgets.IntSlider(min=-5, max=20, step=1, value=20, style=style, layout=layout),
        min_cop=widgets.FloatSlider(min=2, max=7, step=0.1, value=3.5, style=style, layout=layout),
        max_cop=widgets.FloatSlider(min=3, max=7, step=0.1, value=7.5, style=style, layout=layout),
        num_of_heatpumps_millions=widgets.IntSlider(min=1, max=30, step=1, value=1, style=style, layout=layout)
    )

    # Display the FigureWidget and the interactive widget
    display(fig, interactive_plot_widget)

    # Return the updated DataFrame
    return filtered_data_1_updated

# Call the function with your DataFrame and capture the returned DataFrame
updated_df = interactive_apply_linear_cop(filtered_data)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Electricity Grid Load (GW)',
              'type': 'scatter',
              'uid': '756da5c3-b96e-435c-ab15-9ad3ad435c59',
              'x': array([datetime.datetime(2016, 1, 1, 2, 0),
                          datetime.datetime(2016, 1, 1, 3, 0),
                          datetime.datetime(2016, 1, 1, 4, 0), ...,
                          datetime.datetime(2023, 4, 26, 14, 0),
                          datetime.datetime(2023, 4, 26, 15, 0),
                          datetime.datetime(2023, 4, 26, 16, 0)], dtype=object),
              'y': array([0.80624452, 0.80624452, 0.80624452, ..., 0.19166054, 0.19166054,
                                 nan])},
             {'mode': 'lines',
              'name': 'Temp',
              'type': 'scatter',
              'uid': '707fff5d-0e23-41f0-8713-e539aaa301b2',
              'x': array([datetime.datetime(2016, 1, 1, 2, 0),
                          datetime.datetime(2016, 1, 

interactive(children=(IntSlider(value=-5, description='min_temp', layout=Layout(width='500px'), max=20, min=-5…

In [40]:
#filtered_data.to_csv('Heat_pump_load_CET_12000KWh.csv')